<a href="https://colab.research.google.com/github/AaronThomasJoseph/DL-Projects/blob/main/CS671_A1_G21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import math

# **PERCEPTRON**

##**`Classification`**

In [ ]:
# plotting the data
def Plot(Data,title,nrow,ncol,index):
  plt.subplot(nrow,ncol,index)
  color = ["red","green","blue"]
  plt.title(title)
  for i in range(len(Data)):
    plt.scatter(Data[i].T[0],Data[i].T[1],color=color[i],label="Class "+str(i+1),marker='.')
  plt.xlabel("x1")
  plt.ylabel("x2")
  plt.grid()
  plt.legend(loc=1)

In [ ]:
# normalizing the data
def Normalization(Data):
  Data = (Data - Data.min(axis=0)) / (Data.max(axis=0) - Data.min(axis=0))
  return Data

In [ ]:
# confusion matrix
def Confusion_Matrix(Actual,Predicted):
  C = len(set(Actual))
  Matrix = np.zeros((C,C),int)

  for i in range(1,C+1):
    for j in range(1,C+1):
      Matrix[i-1][j-1] = sum(Actual[Predicted==j]==i)

  Accuracy = 0
  Precision= 0
  Recall   = 0
  F1_score = 0
  for i in range(C):
      Accuracy += Matrix[i][i]/Matrix.sum()
      if sum(Matrix[i]):
          Precision+= Matrix[i][i]/sum(Matrix[i])*1/C
      if sum(Matrix[:,i]):
          Recall+=Matrix[i][i]/sum(Matrix[:,i])*1/C

  try:F1_score = 2*(Precision*Recall)/(Precision+Recall)
  except:F1_score = 0

  print('Confusion Matrix:\n',Matrix)
  print('Accuracy:\t',np.around(Accuracy,4))
  print('Precision:\t',np.around(Precision,4))
  print('Recall:\t\t',np.around(Recall,4))
  print('F1-Score:\t',np.around(F1_score,4))

In [ ]:
# activation function for perceptron
def Activation_Function(a,beta=1,func="Logistic"):
  e = np.e
  if func=="Logistic":
    f = 1/(1+e**(-beta*a))
  elif func=="Tanh":
    f = (e**(beta*a) - e**(-beta*a)) / (e**(beta*a) + e**(-beta*a))
  elif func=="Linear":
    f = a
  else:
    return "Wrong Input"
  return f

# gradient of activation function for perceptron
def Gradient_Activation_Function(a,beta=1,func="Logistic"):
  f = Activation_Function(a,beta,func)
  if func=="Logistic":
    df = beta*f*(1-f)
  elif func=="Tanh":
    df = beta*(1-f*f)
  elif func=="Linear":
    df = 1
  else:
    return "Wrong Input"
  return df

In [ ]:
# decision regions
def Decision_Region(C,clf,func,par):
  x1 = np.arange(0,1,0.01)
  x2 = np.arange(0,1,0.01)
  X = np.zeros((len(x1)*len(x2),2))
  for i in range(len(x1)):
    for j in range(len(x2)):
      arr = [x1[i],x2[j]]
      X[i*len(x1)+j] = arr
  y = np.zeros((len(x1)*len(x2)))
  if clf == "perceptron":
    for i in range(len(X)):
        #x = [x0,x1,x2,....]
        x = np.array([1,X[i][0],X[i][1]])
        Pred = []
        #One vs One Approach
        l = 0
        for j in range(C):
          for k in range(j+1,C):
            #Activation Value
            a = par[l]@x
            #Output From Activation Function
            beta=1
            sn = Activation_Function(a,beta,func)
            if func=="Logistic":
              if   sn>0.5:Pred.append(j+1)
              else:       Pred.append(k+1)
            elif func=="Tanh":
              if   sn>0 :Pred.append(j+1)
              else:      Pred.append(k+1)
            l+=1

        lst = Pred
        y[i] = max(set(lst), key=lst.count)

  elif clf == "1 layer":
    W1, b1, W2, b2 , Z1, Z2, A1, A2 = par
    Z1 = W1.dot(X.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    y = get_predictions(A2)+1

  elif clf =="2 layer":
    W1, b1, W2, b2, W3, b3 , Z1, Z2, Z3, A1, A2, A3 = par
    Z1 = W1.dot(X.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    y = get_predictions(A3)+1

  plt.figure(figsize=(10,10))
  plt.title("Decision Region",fontweight="bold")
  Colour = ["salmon","lightgreen","skyblue"]
  for i in range(C):
    x = X[y==i+1]
    plt.scatter(x.T[0],x.T[1],color=Colour[i],label="Surface "+str(i+1),alpha=1/4)
  plt.xlabel("x1")
  plt.ylabel("x2")
  plt.grid()
  plt.legend()
  #plt.tight_layout()
  #plt.show()


In [ ]:
def Perceptron_Classification(X_train,X_validation,X_test,y_train,y_validation,y_test,
               epoch=1e3,error=1e-5,beta=1,func="Logistic",title="Linearly Seperable"):
  #Size of Traininig Data
  N = X_train.shape[0]
  #Size of Validation Data
  M = X_validation.shape[0]
  #Dimension of Input Data
  d = X_train.shape[1]
  #No of Classes
  C = len(set(y_train))
  #Weight Initialization
  w = np.random.random(size=(C*(C-1)//2,d+1))
  #Learning Rate
  learning_rate=1
  #Error
  E_train      = 1e5
  E_train_prev = 0
  #Epoch
  itr = 0
  #Index
  index = list(range(N))
  #To store Training and Validation Errors
  Training_Error  =[]
  Validation_Error=[]
  Test_Error  =[]
  #Perceptron Algorithm
  while((itr<=epoch) and (abs(E_train-E_train_prev)>=error)):
    #Shuffling Training Data
    np.random.shuffle(index)
    X_train = X_train[index]
    y_train = y_train[index]

    E_train_prev = E_train
    E_train      = 0
    for i in range(N):
      #x = [x0,x1,x2,....]
      x = np.concatenate([[1,*X_train[i]]])
      y = y_train[i]
      #One vs One Approach
      l = 0
      for j in range(C):
        for k in range(j+1,C):
          if y==j+1 or y==k+1:
            #Activation Value
            a = w[l]@x
            #Output From Activation Function
            sn = Activation_Function(a,beta,func)
            if func=="Logistic":
              if   y==j+1:yn=1
              elif y==k+1:yn=0
            elif func=="Tanh":
              if   y==j+1:yn= 1
              elif y==k+1:yn=-1

            #En
            E_train += 0.5*(yn-sn)**2
            #𝛿0
            delta0 = (yn-sn)*Gradient_Activation_Function(a,beta,func)
            #w = w + △w  = w + η*𝛿0*x
            w[l] += learning_rate*delta0*x
          l+=1

    E_validation = 0
    for i in range(M):
      #x = [x0,x1,x2,....]
      x = np.concatenate([[1,*X_validation[i]]])
      y = y_validation[i]
      #One vs One Approach
      l = 0
      for j in range(C):
        for k in range(j+1,C):
          if y==j+1 or y==k+1:
            #Activation Value
            a = w[l]@x
            #Output From Activation Function
            sn = Activation_Function(a,beta,func)
            if func=="Logistic":
              if   y==j+1:yn=1
              elif y==k+1:yn=0
            elif func=="Tanh":
              if   y==j+1:yn= 1
              elif y==k+1:yn=-1

            E_validation += 0.5*(yn-sn)**2
          l+=1

    E_test = 0
    for i in range(len(X_test)):
      #x = [x0,x1,x2,....]
      x = np.concatenate([[1,*X_test[i]]])
      y = y_test[i]
      #One vs One Approach
      l = 0
      for j in range(C):
        for k in range(j+1,C):
          if y==j+1 or y==k+1:
            #Activation Value
            a = w[l]@x
            #Output From Activation Function
            sn = Activation_Function(a,beta,func)
            if func=="Logistic":
              if   y==j+1:yn=1
              elif y==k+1:yn=0
            elif func=="Tanh":
              if   y==j+1:yn= 1
              elif y==k+1:yn=-1

            E_test += 0.5*(yn-sn)**2
          l+=1

    learning_rate = 1/(itr+1)**(1/2)
    E_train /= N
    E_validation /= M
    E_test /= len(X_test)
    Training_Error.append(E_train)
    Validation_Error.append(E_validation)
    Test_Error.append(E_test)
    itr+=1

  #Error Plot
  plt.figure(figsize=(12.5,2.5))
  plt.suptitle(title,fontweight="bold",y=1+1e-2)
  plt.subplot(1,3,1)
  plt.title("Training Error")
  plt.plot(range(1,len(Training_Error)+1),Training_Error,marker=".")
  plt.xlabel("Epochs")
  plt.ylabel("Average Error")
  plt.grid()

  # Validation error plot
  plt.subplot(1,3,2)
  plt.title("Validation Error")
  plt.plot(range(1,len(Validation_Error)+1),Validation_Error,marker=".")
  plt.xlabel("Epochs")
  plt.ylabel("Average Error")
  plt.grid()

  # Testing error plot
  plt.subplot(1,3,3)
  plt.title("Test Error")
  plt.plot(range(1,len(Test_Error)+1),Test_Error,marker=".")
  plt.xlabel("Epochs")
  plt.ylabel("Average Error")
  plt.grid()
  plt.tight_layout()
  plt.show()


  # plotting the main points of the data
  l = 0
  x = np.arange(0,1,0.01)
  colour=["red","green","blue"]
  plt.figure(figsize=(12.5,(C*(C+1)//2+1)//2*2.5))
  for j in range(C):
        for k in range(j+1,C):
          X_j = X_train[y_train==j+1]
          X_k = X_train[y_train==k+1]
          plt.subplot((C*(C+1)//2+1)//2,2,l+1)
          plt.title("Class "+str(j+1)+"vs Class "+str(k+1))
          plt.scatter(X_j.T[0],X_j.T[1],color=colour[j],label="Class "+str(j+1),marker='.')
          plt.scatter(X_k.T[0],X_k.T[1],color=colour[k],label="Class "+str(k+1),marker='.')
          y = -(w[l][1]*x + w[l][0])/w[l][2]
          plt.plot(x,y,label="Decision\nBoundary",color="black",linestyle="-.")
          plt.xlabel("x1")
          plt.ylabel("y1")
          plt.xlim(-0.05,1.05)
          plt.ylim(-0.05,1.05)
          plt.legend(loc=1,fontsize="small",framealpha=0)
          plt.grid()
          l+=1
  plt.tight_layout()
  plt.show()

  Decision_Region(C,clf="perceptron",func="Logistic",par=w)
  for k in range(C):
    X = X_train[y_train==k+1]
    plt.scatter(X.T[0],X.T[1],color=colour[k],label="Class "+str(j+1),marker='.')

  plt.tight_layout()
  plt.show()
  #Prediction
  #Training Data
  y_train_pred = y_train*0
  for i in range(N):
    #x = [x0,x1,x2,....]
    x = np.concatenate([[1,*X_train[i]]])
    Pred = []
    #One vs One Approach
    l = 0
    for j in range(C):
      for k in range(j+1,C):
        #Activation Value
        a = w[l]@x
        #Output From Activation Function
        sn = Activation_Function(a,beta,func)
        if func=="Logistic":
          if   sn>0.5:Pred.append(j+1)
          else:       Pred.append(k+1)
        elif func=="Tanh":
          if   sn>0 :Pred.append(j+1)
          else:      Pred.append(k+1)
        l+=1

    y_train_pred[i] = statistics.mode(Pred)
    Pred = []

  #Validation Data
  y_validation_pred = y_validation*0
  for i in range(M):
    #x = [x0,x1,x2,....]
    x = np.concatenate([[1,*X_validation[i]]])
    Pred = []
    #One vs One Approach
    l = 0
    for j in range(C):
      for k in range(j+1,C):
        #Activation Value
        a = w[l]@x
        #Output From Activation Function
        beta=1
        sn = Activation_Function(a,beta,func)
        if func=="Logistic":
          if   sn>0.5:Pred.append(j+1)
          else:       Pred.append(k+1)
        elif func=="Tanh":
          if   sn>0 :Pred.append(j+1)
          else:      Pred.append(k+1)
        l+=1

    y_validation_pred[i] = statistics.mode(Pred)
    Pred = []

  #Test Data
  y_test_pred = y_test*0
  for i in range(len(X_test)):
    #x = [x0,x1,x2,....]
    x = np.concatenate([[1,*X_test[i]]])
    Pred = []
    #One vs One Approach
    l = 0
    for j in range(C):
      for k in range(j+1,C):
        #Activation Value
        a = w[l]@x
        #Output From Activation Function
        sn = Activation_Function(a,beta,func)
        if func=="Logistic":
          if   sn>0.5:Pred.append(j+1)
          else:       Pred.append(k+1)
        elif func=="Tanh":
          if   sn>0 :Pred.append(j+1)
          else:      Pred.append(k+1)
        l+=1

    y_test_pred[i] = statistics.mode(Pred)
    Pred = []

  print('*'*80)
  print(" "*35+"Training Data")
  print(" "*35+"_____________")
  Confusion_Matrix(y_train,y_train_pred)
  print("\n"+" "*35+"Validation Data")
  print(" "*35+"_______________")
  Confusion_Matrix(y_validation,y_validation_pred)
  print("\n"+" "*35+"Test Data")
  print(" "*35+"_________")
  Confusion_Matrix(y_test,y_test_pred)
  print('*'*80)

In [ ]:
#DataSet
#Linearly Seperable Data
LS_C1 = np.loadtxt("LS_Class1.txt")
LS_C2 = np.loadtxt("LS_Class2.txt")
LS_C3 = np.loadtxt("LS_Class3.txt")

#Min_Max Normalization
LS = np.concatenate([LS_C1,LS_C2,LS_C3])
LS = Normalization(LS)
LS_C1,LS_C2,LS_C3 = LS[:len(LS_C1)],LS[len(LS_C1):len(LS_C1)+len(LS_C2)],LS[len(LS_C1)+len(LS_C2):]

#Plot
plt.figure(figsize=(12,12.5))
plt.suptitle("Classification Data",fontweight="bold",y=1+1e-2)
Plot(Data=[LS_C1,LS_C2,LS_C3],title="Linearly Seperable Data",nrow=4,ncol=2,index=1)

#Train-Validation-Test Split
np.random.shuffle(LS_C1)
np.random.shuffle(LS_C2)
np.random.shuffle(LS_C3)
len1 = len(LS_C1)
len2 = len(LS_C2)
len3  = len(LS_C3)
LS_C1_train,LS_C1_validation,LS_C1_test = LS_C1[:int(.60*len1)],LS_C1[int(.60*len1):int(.80*len1)],LS_C1[int(.80*len1):]
LS_C2_train,LS_C2_validation,LS_C2_test = LS_C2[:int(.60*len2)],LS_C2[int(.60*len2):int(.80*len2)],LS_C2[int(.80*len2):]
LS_C3_train,LS_C3_validation,LS_C3_test = LS_C3[:int(.60*len3)],LS_C3[int(.60*len3):int(.80*len3)],LS_C3[int(.80*len3):]

LS_X_train = np.concatenate((LS_C1_train,LS_C2_train,LS_C3_train))
LS_y_train = np.concatenate((1*np.ones(len(LS_C1_train)),2*np.ones(len(LS_C2_train)),3*np.ones(len(LS_C3_train))))
LS_X_validation = np.concatenate((LS_C1_validation,LS_C2_validation,LS_C3_validation))
LS_y_validation = np.concatenate((1*np.ones(len(LS_C1_validation)),2*np.ones(len(LS_C2_validation)),3*np.ones(len(LS_C3_validation))))
LS_X_test = np.concatenate((LS_C1_test,LS_C2_test,LS_C3_test))
LS_y_test = np.concatenate((1*np.ones(len(LS_C1_test)),2*np.ones(len(LS_C2_test)),3*np.ones(len(LS_C3_test))))

#Plot
Plot(Data=[LS_C1_train,LS_C2_train,LS_C3_train],title="Linearly Seperable Training Data",nrow=4,ncol=2,index=3)
Plot(Data=[LS_C1_validation,LS_C2_validation,LS_C3_validation],title="Linearly Seperable Validation Data",nrow=4,ncol=2,index=5)
Plot(Data=[LS_C1_test,LS_C2_test,LS_C3_test],title="Linearly Seperable Test Data",nrow=4,ncol=2,index=7)

#Non Linearly Seperable Data
NLS = np.loadtxt("NLS_Group21.txt",skiprows=1,)
NLS_C1 = NLS[:2446]
NLS_C2 = NLS[2446:]

#Min-Max Normalization
NLS = Normalization(NLS)
NLS_C1,NLS_C2 = NLS[:len(NLS_C1)],NLS[len(NLS_C1):]

#Plot
Plot(Data=[NLS_C1,NLS_C2],title="Non Linearly Seperable Data",nrow=4,ncol=2,index=2)

#Train-Validation-Test Split
np.random.shuffle(NLS_C1)
np.random.shuffle(NLS_C2)
len1 = len(NLS_C1)
len2 = len(NLS_C2)
NLS_C1_train,NLS_C1_validation,NLS_C1_test = NLS_C1[:int(.60*len1)],NLS_C1[int(.60*len1):int(.80*len1)],NLS_C1[int(.80*len1):]
NLS_C2_train,NLS_C2_validation,NLS_C2_test = NLS_C2[:int(.60*len2)],NLS_C2[int(.60*len2):int(.80*len2)],NLS_C2[int(.80*len2):]

NLS_X_train = np.concatenate((NLS_C1_train,NLS_C2_train))
NLS_y_train = np.concatenate((1*np.ones(len(NLS_C1_train)),2*np.ones(len(NLS_C2_train))))
NLS_X_validation = np.concatenate((NLS_C1_validation,NLS_C2_validation))
NLS_y_validation = np.concatenate((1*np.ones(len(NLS_C1_validation)),2*np.ones(len(NLS_C2_validation))))
NLS_X_test = np.concatenate((NLS_C1_test,NLS_C2_test))
NLS_y_test = np.concatenate((1*np.ones(len(NLS_C1_test)),2*np.ones(len(NLS_C2_test))))

#Plot
Plot(Data=[NLS_C1_train,NLS_C2_train],title="Non Linearly Seperable Training Data",nrow=4,ncol=2,index=4)
Plot(Data=[NLS_C1_validation,NLS_C2_validation],title="Non Linearly Seperable Validation Data",nrow=4,ncol=2,index=6)
Plot(Data=[NLS_C1_test,NLS_C2_test],title="Non Linearly Seperable Test Data",nrow=4,ncol=2,index=8)

plt.tight_layout()
plt.show()

#Perceptron Classification
#Linearly Seperable Data
Perceptron_Classification(LS_X_train,LS_X_validation,LS_X_test,LS_y_train,LS_y_validation,LS_y_test,title="Linearly Seperable Data")

#Non Linearly Seperable Data
Perceptron_Classification(NLS_X_train,NLS_X_validation,NLS_X_test,NLS_y_train,NLS_y_validation,NLS_y_test,title="Non Linearly Seperable Data")



##**`Regression`**

In [ ]:
def Perceptron_Regression(X_train,X_validation,X_test,y_train,y_validation,y_test,
               epoch=1e3,error=1e-5,beta=1,func="Linear",title="Linearly Seperable"):
  #Size of Traininig Data
  N = X_train.shape[0]
  #Size of Validation Data
  M = X_validation.shape[0]
  #Dimension of Input Data
  try:d = X_train.shape[1]
  except:d=1
  #Weight Initialization
  w = np.random.random(size=(d+1))
  #Learning Rate
  learning_rate=1
  #Error
  E_train      = 1e5
  E_train_prev = 0
  #Epoch
  itr = 0
  #Index
  index = list(range(N))
  #To store Training and Validation Errors
  Training_Error  =[]
  Validation_Error=[]
  Test_Error      =[]
  #Perceptron Algorithm
  while((itr<=epoch) and (abs(E_train-E_train_prev)>=error)):
    #Shuffling Training Data
    np.random.shuffle(index)
    X_train = X_train[index]
    y_train = y_train[index]

    E_train_prev = E_train
    E_train      = 0
    for i in range(N):
      #x = [x0,x1,x2,....]
      try:x = np.concatenate([[1,*X_train[i]]])
      except:x = np.array([1,X_train[i]])
      y = y_train[i]

      #Activation Value
      a = w@x
      #Output From Activation Function
      #Activation Function is the Linear Function
      sn = Activation_Function(a,beta,func)

      #En
      E_train += 0.5*(y-sn)**2
      #𝛿0
      delta0 = (y-sn)*Gradient_Activation_Function(a,beta,func)
      #w = w + △w  = w + η*𝛿0*x
      w += learning_rate*delta0*x


    E_validation = 0
    for i in range(M):
      #x = [x0,x1,x2,....]
      try:x = np.concatenate([[1,*X_validation[i]]])
      except:x = np.array([1,X_validation[i]])
      y = y_validation[i]

      #Activation Value
      a = w@x
      #Output From Activation Function
      sn = Activation_Function(a,beta,func)

      E_validation += 0.5*(y-sn)**2

    E_test = 0
    for i in range(len(X_test)):
      #x = [x0,x1,x2,....]
      try:x = np.concatenate([[1,*X_test[i]]])
      except:x = np.array([1,X_test[i]])
      y = y_test[i]

      #Activation Value
      a = w@x
      #Output From Activation Function
      sn = Activation_Function(a,beta,func)

      E_test += 0.5*(y-sn)**2

    learning_rate = 1/(itr+1)**(1/2)
    E_train /= 2*N
    E_validation /= 2*M
    E_test /= 2*len(X_test)
    Training_Error.append(E_train)
    Validation_Error.append(E_validation)
    Test_Error.append(E_test)
    itr+=1

  #Error Plot
  plt.figure(figsize=(12,5))
  plt.suptitle(title,fontweight="bold",y=1+1e-2)
  plt.subplot(1,3,1)
  plt.title("Training Error")
  plt.plot(range(1,len(Training_Error)+1),Training_Error,marker=".")
  plt.xlabel("Epochs")
  plt.ylabel("Average Error")
  plt.grid()

  plt.subplot(1,3,2)
  plt.title("Validation Error")
  plt.plot(range(1,len(Validation_Error)+1),Validation_Error,marker=".")
  plt.xlabel("Epochs")
  plt.ylabel("Average Error")
  plt.grid()
  #plt.tight_layout()


  plt.subplot(1,3,3)
  plt.title("Test Error")
  plt.plot(range(1,len(Test_Error)+1),Test_Error,marker=".")
  plt.xlabel("Epochs")
  plt.ylabel("Average Error")
  plt.grid()
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12,5))
  plt.suptitle(title,fontweight="bold",y=1+1e-2)
  bar_x = ("Training Data","Validation Data","Test Data")
  bar_y = (Training_Error[-1],Validation_Error[-1],Test_Error[-1])
  plt.bar(bar_x,bar_y)
  plt.xlabel("Data")
  plt.ylabel("Mean Square Error")
  plt.grid()
  plt.tight_layout()
  plt.show()
  #Prediction
  #Training Data
  y_train_pred = y_train*0
  for i in range(N):
    #x = [x0,x1,x2,....]
    try:x = np.concatenate([[1,*X_train[i]]])
    except:x = np.array([1,X_train[i]])
    #Activation Value
    a = w@x
    #Output From Activation Function
    sn = Activation_Function(a,beta,func)
    y_train_pred[i] = sn

  #Validation Data
  y_validation_pred = y_validation*0
  for i in range(M):
    #x = [x0,x1,x2,....]
    try:x = np.concatenate([[1,*X_validation[i]]])
    except:x = np.array([1,X_validation[i]])
    #Activation Value
    a = w@x
    #Output From Activation Function
    sn = Activation_Function(a,beta,func)
    y_validation_pred[i] = sn

  #Test Data
  y_test_pred = y_test*0
  for i in range(len(X_test)):
    #x = [x0,x1,x2,....]
    try:x = np.concatenate([[1,*X_test[i]]])
    except:x = np.array([1,X_test[i]])
    #Activation Value
    a = w@x
    #Output From Activation Function
    sn = Activation_Function(a,beta,func)
    y_test_pred[i] = sn

  #Actual vs Predicted Output
  plt.figure(figsize=(12,5))
  plt.suptitle(title,fontweight="bold",y=1+1e-2)
  #Training Data
  plt.subplot(1,3,1)
  plt.title("Training Data")
  plt.scatter(y_train,y_train_pred,marker=".")
  plt.xlabel("Target Output")
  plt.ylabel("Model Output")
  plt.grid()
  #Validation Data
  plt.subplot(1,3,2)
  plt.title("Validation Data")
  plt.scatter(y_validation,y_validation_pred,marker=".")
  plt.xlabel("Target Output")
  plt.ylabel("Model Output")
  plt.grid()
  #Test Data
  plt.subplot(1,3,3)
  plt.title("Training Data")
  plt.scatter(y_test,y_test_pred,marker=".")
  plt.xlabel("Target Output")
  plt.ylabel("Model Output")
  plt.grid()
  plt.tight_layout()
  plt.show()

  return (X_train,X_validation,X_test),(y_train,y_validation,y_test),(y_train_pred,y_validation_pred,y_test_pred)

In [ ]:
#DataSet
#Univariate Data
Univariate_Data = pd.read_csv("UnivariateData.csv",header=None).values

#Plot
plt.figure(figsize=(12,12.5))
plt.suptitle("Regression Data",fontweight="bold",y=1+1e-2)
plt.subplot(4,2,1)
plt.title("Univariate Data")
plt.scatter(Univariate_Data.T[0],Univariate_Data.T[1])
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

#Train-Validation-Test Split
np.random.shuffle(Univariate_Data)
len_univariate  = len(Univariate_Data)
Univariate_train,Univariate_validation,Univariate_test = Univariate_Data[:int(.60*len_univariate)],Univariate_Data[int(.60*len_univariate):int(.80*len_univariate)],Univariate_Data[int(.80*len_univariate):]


#Plot
plt.subplot(4,2,3)
plt.title("Univariate Training Data")
plt.scatter(Univariate_train.T[0],Univariate_train.T[1])
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

plt.subplot(4,2,5)
plt.title("Univariate Validation Data")
plt.scatter(Univariate_validation.T[0],Univariate_validation.T[1])
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

plt.subplot(4,2,7)
plt.title("Univariate Test Data")
plt.scatter(Univariate_test.T[0],Univariate_test.T[1])
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

#Bivariate Data
Bivariate_Data = pd.read_csv("BivariateData.csv",header=None).values

#Train-Validation-Test Split
np.random.shuffle(Bivariate_Data)
len_bivariate  = len(Bivariate_Data)
Bivariate_train,Bivariate_validation,Bivariate_test = Bivariate_Data[:int(.60*len_bivariate)],Bivariate_Data[int(.60*len_bivariate):int(.80*len_bivariate)],Bivariate_Data[int(.80*len_bivariate):]

#Plot
ax = plt.subplot(4,2,2,projection="3d")
plt.title("Bivariate Data")
ax.scatter3D(*Bivariate_Data.T)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()

ax = plt.subplot(4,2,4,projection="3d")
plt.title("Bivariate Training Data")
ax.scatter3D(*Bivariate_train.T)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()

ax = plt.subplot(4,2,6,projection="3d")
plt.title("Bivariate Validation Data")
ax.scatter3D(*Bivariate_validation.T)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()

ax = plt.subplot(4,2,8,projection="3d")
plt.title("Bivariate Test Data")
ax.scatter3D(*Bivariate_test.T)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()

plt.tight_layout()
plt.show()

#Perceptron Regression
#Univariate Data
Univariate_X,Univariate_y,Univariate_y_pred = Perceptron_Regression(Univariate_train.T[0],Univariate_validation.T[0],Univariate_test.T[0],
                                            Univariate_train.T[-1],Univariate_validation.T[-1],Univariate_test.T[-1],title="Univariate Data")
#Bivariate Data
Bivariate_X,Bivariate_y,Bivariate_y_pred = Perceptron_Regression(Bivariate_train.T[:2].T,Bivariate_validation.T[:2].T,Bivariate_test.T[:2].T,
                                            Bivariate_train.T[-1],Bivariate_validation.T[-1],Bivariate_test.T[-1],title="Bivariate Data")

#Plot
plt.figure(figsize=(12,12.5))
plt.suptitle("Actual vs Predicted Output",fontweight="bold",y=1+1e-2)
plt.subplot(3,2,1)
plt.title("Univariate Train Data")
plt.scatter(Univariate_X[0],Univariate_y[0],label='Actual',alpha=1/2)
plt.scatter(Univariate_X[0],Univariate_y_pred[0],label='Prediction',alpha=1/2)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

plt.subplot(3,2,3)
plt.title("Univariate Validation Data")
plt.scatter(Univariate_X[1],Univariate_y[1],label='Actual',alpha=1/2)
plt.scatter(Univariate_X[1],Univariate_y_pred[1],label='Prediction',alpha=1/2)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

plt.subplot(3,2,5)
plt.title("Univariate Test Data")
plt.scatter(Univariate_X[2],Univariate_y[2],label='Actual',alpha=1/2)
plt.scatter(Univariate_X[2],Univariate_y_pred[2],label='Prediction',alpha=1/2)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,2,projection="3d")
plt.title("Bivariate Train Data")
ax.scatter3D(*Bivariate_X[0].T,Bivariate_y[0],label='Actual',alpha=1/2)
ax.scatter3D(*Bivariate_X[0].T,Bivariate_y_pred[0],label='Prediction',alpha=1/2)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,4,projection="3d")
plt.title("Bivariate Validation Data")
ax.scatter3D(*Bivariate_X[1].T,Bivariate_y[1],label='Actual',alpha=1/2)
ax.scatter3D(*Bivariate_X[1].T,Bivariate_y_pred[1],label='Prediction',alpha=1/2)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,6,projection="3d")
plt.title("Bivariate Test Data")
ax.scatter3D(*Bivariate_X[2].T,Bivariate_y[2],label='Actual',alpha=1/2)
ax.scatter3D(*Bivariate_X[2].T,Bivariate_y_pred[2],label='Prediction',alpha=1/2)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)
plt.tight_layout()
plt.show()

# **FCNN**

##Classification

### 1 hidden layer

In [ ]:
def init_params(d,C,nodes):
    W1 = np.random.normal(0,1,(nodes, d))
    b1 = np.random.normal(0,1,(nodes, 1))
    W2 = np.random.normal(0,1,(C, nodes))
    b2 = np.random.normal(0,1,(C, 1))
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    X = np.reshape(X, (2, 1))
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y,C):
    try:
      vec = np.zeros(C,int)
      vec[int(Y)-1] = 1
    except:
      vec = np.zeros((len(Y),C),int)
      for i in range(C):
        vec[Y==i+1,i] = 1
    return vec.T

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y,d,C,nodes):
    one_hot_Y = one_hot(Y,C)
    one_hot_Y = np.reshape(one_hot_Y, (C, 1))
    X = np.reshape(X,(d,1))
    dZ2 = A2 - one_hot_Y
    dZ2 = np.reshape(dZ2, (C, 1))
    dW2 = dZ2.dot(A1.T)
    dW2 = np.reshape(dW2, (C, nodes))
    db2 = np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = dZ1.dot(X.T)
    db1 = np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2

    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y,X_validation,Y_validation,X_test,Y_test,iterations,alpha,Error,nodes,title):
    m = X.shape[0]
    d = X.shape[1]
    C = len(set(Y))
    Error_train      = []
    Error_validation = []
    Error_test       = []
    W1, b1, W2, b2 = init_params(d,C,nodes)
    for i in range(int(iterations)):
      for j in range(m):
        k = np.random.randint(0,m-1)
        tempX = X[k]
        tempY = Y[k]

        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, tempX)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, tempX, tempY,d,C,nodes)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)


      Z1 = W1.dot(X.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = softmax(Z2)
      y = one_hot(Y,C)
      Error_train.append(((A2-y)**2).sum() / m)
      predictions_train = get_predictions(A2)

      Z1 = W1.dot(X_validation.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = softmax(Z2)
      y = one_hot(Y_validation,C)
      Error_validation.append(((A2-y)**2).sum() / m)
      predictions_validation = get_predictions(A2)

      Z1 = W1.dot(X_test.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = softmax(Z2)
      y = one_hot(Y_test,C)
      Error_test.append(((A2-y)**2).sum() / m)
      predictions_test = get_predictions(A2)

      if i>1:
        if abs(Error_train[-1]-Error_train[-2]) and abs(Error_train[-1]-Error_train[-3]) <Error:
          break
    #Decision Region
    Decision_Region(C,clf="1 layer",func=None,par=(W1, b1, W2, b2 , Z1, Z2, A1, A2))
    Colour=["red","green","blue"]
    for k in range(C):
      x = X[Y==k+1]
      plt.scatter(x.T[0],x.T[1],color=Colour[k],label="Class "+str(j+1),marker='.')
    plt.tight_layout()
    plt.show()

    #Error Plot
    plt.figure(figsize=(12.5,5))
    plt.suptitle(title,fontweight="bold",y=1+1e-2)
    plt.subplot(1,3,1)
    plt.title("Training Error")
    plt.plot(range(1,len(Error_train )+1),Error_train ,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()

    plt.subplot(1,3,2)
    plt.title("Validation Error")
    plt.plot(range(1,len(Error_validation)+1),Error_validation,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()

    plt.subplot(1,3,3)
    plt.title("Test Error")
    plt.plot(range(1,len(Error_test)+1),Error_test,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()
    plt.tight_layout()
    plt.show()

    print('*'*80)
    print(" "*35+"Training Data")
    print(" "*35+"_____________")
    Confusion_Matrix(Y,predictions_train+1)
    print("\n"+" "*35+"Validation Data")
    print(" "*35+"_______________")
    Confusion_Matrix(Y_validation,predictions_validation+1)
    print("\n"+" "*35+"Test Data")
    print(" "*35+"_________")
    Confusion_Matrix(Y_test,predictions_test+1)
    print('*'*80)

    #Plot Hidden Layer
    plt.figure(figsize=(12.5,4*nodes))
    plt.suptitle(title+"\nHidden Layer",fontweight="bold",y=1+1e-2)

    # plots of hidden nodes
    p = 1
    Z1 = W1.dot(X.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    Colour=["red","green","blue"]
    for j in range(nodes):
      ax = plt.subplot(nodes,3,p,projection="3d")
      plt.title("Training Data\nHidden Node"+str(j+1))
      for i in range(C):
        x  = X[Y==i+1]
        a1 = A1[j,Y==i+1]
        ax.scatter3D(x.T[0],x.T[1],a1,color=Colour[i],label="Class "+str(i+1))
      plt.xlabel("x1")
      plt.ylabel("x2")
      ax.set_zlabel("Activation for Neuron "+str(j+1))
      plt.grid()
      plt.legend(loc=1,fontsize="small")
      p+=3

    p=2
    Z1 = W1.dot(X_validation.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    Colour=["red","green","blue"]
    for j in range(nodes):
      ax = plt.subplot(nodes,3,p,projection="3d")
      plt.title("Validation Data\nHidden Node"+str(j+1))
      for i in range(C):
        x  = X_validation[Y_validation==i+1]
        a1 = A1[j,Y_validation==i+1]
        ax.scatter3D(x.T[0],x.T[1],a1,color=Colour[i],label="Class "+str(i+1))
      plt.xlabel("x1")
      plt.ylabel("x2")
      ax.set_zlabel("Activation for Neuron "+str(j+1))
      plt.grid()
      plt.legend(loc=1,fontsize="small")
      p+=3

    p=3
    Z1 = W1.dot(X_test.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    Colour=["red","green","blue"]
    for j in range(nodes):
      ax = plt.subplot(nodes,3,p,projection="3d")
      plt.title("Test Data\nHidden Node"+str(j+1))
      for i in range(C):
        x  = X_test[Y_test==i+1]
        a1 = A1[j,Y_test==i+1]
        ax.scatter3D(x.T[0],x.T[1],a1,color=Colour[i],label="Class "+str(i+1))
      plt.xlabel("x1")
      plt.ylabel("x2")
      ax.set_zlabel("Activation for Neuron "+str(j+1))
      plt.grid()
      plt.legend(loc=1,fontsize="small")
      p+=3
    #plt.tight_layout()
    plt.show()

    #Plot Output Layer
    plt.figure(figsize=(12.5,4*C))
    plt.suptitle(title+"\nOutput Layer",fontweight="bold",y=1+1e-2)

    p = 1
    Z1 = W1.dot(X.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    Colour=["red","green","blue"]
    for j in range(C):
      ax = plt.subplot(C,3,p,projection="3d")
      plt.title("Training Data\nOutput Node"+str(j+1))
      for i in range(C):
        x  = X[Y==i+1]
        a2 = A2[j,Y==i+1]
        ax.scatter3D(x.T[0],x.T[1],a2,color=Colour[i],label="Class "+str(i+1))
      plt.xlabel("x1")
      plt.ylabel("x2")
      ax.set_zlabel("Activation for Neuron "+str(j+1))
      plt.grid()
      plt.legend(loc=1,fontsize="small")
      p+=3

    p=2
    Z1 = W1.dot(X_validation.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    Colour=["red","green","blue"]
    for j in range(C):
      ax = plt.subplot(C,3,p,projection="3d")
      plt.title("Validation Data\nOutput Node"+str(j+1))
      for i in range(C):
        x  = X_validation[Y_validation==i+1]
        a2 = A2[j,Y_validation==i+1]
        ax.scatter3D(x.T[0],x.T[1],a2,color=Colour[i],label="Class "+str(i+1))
      plt.xlabel("x1")
      plt.ylabel("x2")
      ax.set_zlabel("Activation for Neuron "+str(j+1))
      plt.grid()
      plt.legend(loc=1,fontsize="small")
      p+=3

    p=3
    Z1 = W1.dot(X_test.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    Colour=["red","green","blue"]
    for j in range(C):
      ax = plt.subplot(C,3,p,projection="3d")
      plt.title("Test Data\nOutput Node"+str(j+1))
      for i in range(C):
        x  = X_test[Y_test==i+1]
        a2 = A2[j,Y_test==i+1]
        ax.scatter3D(x.T[0],x.T[1],a2,color=Colour[i],label="Class "+str(i+1))
      plt.xlabel("x1")
      plt.ylabel("x2")
      ax.set_zlabel("Activation for Neuron "+str(j+1))
      plt.grid()
      plt.legend(loc=1,fontsize="small")
      p+=3
    #plt.tight_layout()
    plt.show()
    return W1, b1, W2, b2 , Z1, Z2, A1, A2

In [ ]:
W1, b1, W2, b2 , Z1, Z2, A1, A2 = gradient_descent(LS_X_train, LS_y_train,LS_X_validation,LS_y_validation,LS_X_test,LS_y_test,
                                  alpha=0.01,Error=1e-5,iterations=1e3,nodes=3,title="Linear Seperable Data")

In [ ]:
W1, b1, W2, b2 , Z1, Z2, A1, A2 = gradient_descent(NLS_X_train, NLS_y_train,NLS_X_validation,NLS_y_validation,NLS_X_test,NLS_y_test,
                                  alpha=0.01,Error=1e-5,iterations=1e3,nodes=100,title="Non Linear Seperable Data")

### 2 hidden layer

In [ ]:
def init_params(d,C,node1,node2):
    W1 = np.random.normal(0,1,(node1, d))
    b1 = np.random.normal(0,1,(node1, 1))
    W2 = np.random.normal(0,1,(node2,node1))
    b2 = np.random.normal(0,1,(node2, 1))
    W3 = np.random.normal(0,1,(C, node2))
    b3 = np.random.normal(0,1,(C, 1))
    return W1, b1, W2, b2, W3, b3

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    X = np.reshape(X, (2, 1))
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3



def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y,C,d,node1,node2):

    one_hot_Y = one_hot(Y,C)
    one_hot_Y = np.reshape(one_hot_Y, (C, 1))
    X = np.reshape(X,(d,1))

    dZ3 = A3 - one_hot_Y
    dZ3 = np.reshape(dZ3, (C, 1))
    dW3 = dZ3.dot(A2.T)
    dW3 = np.reshape(dW3, (C, node2))
    db3 = np.sum(dZ3)

    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dZ2 = np.reshape(dZ2, (node2, 1))
    dW2 = dZ2.dot(A1.T)
    dW2 = np.reshape(dW2, (node2, node1))
    db2 = np.sum(dZ2)

    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = dZ1.dot(X.T)
    db1 = np.sum(dZ1)
    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    return W1, b1, W2, b2, W3, b3

In [ ]:
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y,X_validation,Y_validation,X_test,Y_test,iterations,alpha,Error,node1,node2,title):
      m = X.shape[0]
      d = X.shape[1]
      C = len(set(Y))
      Error_train      = []
      Error_validation = []
      Error_test       = []
      W1, b1, W2, b2, W3, b3 = init_params(d,C,node1,node2)
      for i in range(int(iterations)):
        for j in range(m):
          k = np.random.randint(0,m-1)
          tempX = X[k]
          tempY = Y[k]

          Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, tempX)

          dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, tempX, tempY,C,d,node1,node2)
          W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)

        Z1 = W1.dot(X.T) + b1
        A1 = ReLU(Z1)
        Z2 = W2.dot(A1) + b2
        A2 = ReLU(Z2)
        Z3 = W3.dot(A2) + b3
        A3 = softmax(Z3)
        y = one_hot(Y,C)
        Error_train.append(((A3-y)**2).sum() / len(X))
        predictions_train = get_predictions(A3)+1

        Z1 = W1.dot(X_validation.T) + b1
        A1 = ReLU(Z1)
        Z2 = W2.dot(A1) + b2
        A2 = ReLU(Z2)
        Z3 = W3.dot(A2) + b3
        A3 = softmax(Z3)
        y = one_hot(Y_validation,C)
        Error_validation.append(((A3-y)**2).sum() / len(X_validation))
        predictions_validation = get_predictions(A3)+1

        Z1 = W1.dot(X_test.T) + b1
        A1 = ReLU(Z1)
        Z2 = W2.dot(A1) + b2
        A2 = ReLU(Z2)
        Z3 = W3.dot(A2) + b3
        A3 = softmax(Z3)
        y = one_hot(Y_test,C)
        Error_test.append(((A3-y)**2).sum() / len(X_test))
        predictions_test = get_predictions(A3)+1

        if i>1:
          if abs(Error_train[-1]-Error_train[-2]) and abs(Error_train[-1]-Error_train[-3]) <Error:
            break
      #Decision Region
      Decision_Region(C,clf="2 layer",func=None,par=(W1, b1, W2, b2,W3,b3 , Z1, Z2,Z3, A1, A2,A3))
      Colour=["red","green","blue"]
      for k in range(C):
        x = X[Y==k+1]
        plt.scatter(x.T[0],x.T[1],color=Colour[k],label="Class "+str(j+1),marker='.')
      plt.tight_layout()
      plt.show()

      #Error Plot
      plt.figure(figsize=(12,5))
      plt.suptitle(title,fontweight="bold",y=1+1e-2)
      plt.subplot(1,3,1)
      plt.title("Training Error")
      plt.plot(range(1,len(Error_train )+1),Error_train ,marker=".")
      plt.xlabel("Epochs")
      plt.ylabel("Average Error")
      plt.grid()

      plt.subplot(1,3,2)
      plt.title("Validation Error")
      plt.plot(range(1,len(Error_validation)+1),Error_validation,marker=".")
      plt.xlabel("Epochs")
      plt.ylabel("Average Error")
      plt.grid()

      plt.subplot(1,3,3)
      plt.title("Test Error")
      plt.plot(range(1,len(Error_test)+1),Error_test,marker=".")
      plt.xlabel("Epochs")
      plt.ylabel("Average Error")
      plt.grid()
      plt.tight_layout()
      plt.show()

      #print(predictions_train)
      #print(predictions_validation)
      #print(predictions_test)
      print('*'*80)
      print(" "*35+"Training Data")
      print(" "*35+"_____________")
      Confusion_Matrix(Y,predictions_train)
      print("\n"+" "*35+"Validation Data")
      print(" "*35+"_______________")
      Confusion_Matrix(Y_validation,predictions_validation)
      print("\n"+" "*35+"Test Data")
      print(" "*35+"_________")
      Confusion_Matrix(Y_test,predictions_test)
      print('*'*80)

      #Plot Hidden Layer1
      plt.figure(figsize=(12.5,4*node1))
      plt.suptitle(title+"\nHidden Layer1",fontweight="bold",y=1+1e-2)

      p = 1
      Z1 = W1.dot(X.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(node1):
        ax = plt.subplot(node1,3,p,projection="3d")
        plt.title("Training Data\nHidden Node"+str(j+1))
        for i in range(C):
          x  = X[Y==i+1]
          a1 = A1[j,Y==i+1]
          ax.scatter3D(x.T[0],x.T[1],a1,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3

      p=2
      Z1 = W1.dot(X_validation.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(node1):
        ax = plt.subplot(node1,3,p,projection="3d")
        plt.title("Validation Data\nHidden Node"+str(j+1))
        for i in range(C):
          x  = X_validation[Y_validation==i+1]
          a1 = A1[j,Y_validation==i+1]
          ax.scatter3D(x.T[0],x.T[1],a1,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3

      p=3
      Z1 = W1.dot(X_test.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(node1):
        ax = plt.subplot(node1,3,p,projection="3d")
        plt.title("Test Data\nHidden Node"+str(j+1))
        for i in range(C):
          x  = X_test[Y_test==i+1]
          a1 = A1[j,Y_test==i+1]
          ax.scatter3D(x.T[0],x.T[1],a1,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3
      #plt.tight_layout()
      plt.show()

      #Plot Hidden Layer2
      plt.figure(figsize=(12.5,4*node2))
      plt.suptitle(title+"\nHidden Layer2",fontweight="bold",y=1+1e-2)

      p = 1
      Z1 = W1.dot(X.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(node2):
        ax = plt.subplot(node2,3,p,projection="3d")
        plt.title("Training Data\nHidden Node"+str(j+1))
        for i in range(C):
          x  = X[Y==i+1]
          a2 = A2[j,Y==i+1]
          ax.scatter3D(x.T[0],x.T[1],a2,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3

      p=2
      Z1 = W1.dot(X_validation.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(node2):
        ax = plt.subplot(node2,3,p,projection="3d")
        plt.title("Validation Data\nHidden Node"+str(j+1))
        for i in range(C):
          x  = X_validation[Y_validation==i+1]
          a2 = A2[j,Y_validation==i+1]
          ax.scatter3D(x.T[0],x.T[1],a2,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3

      p=3
      Z1 = W1.dot(X_test.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(node2):
        ax = plt.subplot(node2,3,p,projection="3d")
        plt.title("Test Data\nHidden Node"+str(j+1))
        for i in range(C):
          x  = X_test[Y_test==i+1]
          a2 = A2[j,Y_test==i+1]
          ax.scatter3D(x.T[0],x.T[1],a2,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3
      #plt.tight_layout()
      plt.show()

      #Plot Output Layer
      plt.figure(figsize=(12.5,4*C))
      plt.suptitle(title+"\nOutput Layer",fontweight="bold",y=1+1e-2)

      p = 1
      Z1 = W1.dot(X.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(C):
        ax = plt.subplot(C,3,p,projection="3d")
        plt.title("Training Data\nOutput Node"+str(j+1))
        for i in range(C):
          x  = X[Y==i+1]
          a3 = A3[j,Y==i+1]
          ax.scatter3D(x.T[0],x.T[1],a3,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3

      p=2
      Z1 = W1.dot(X_validation.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(C):
        ax = plt.subplot(C,3,p,projection="3d")
        plt.title("Validation Data\nOutput Node"+str(j+1))
        for i in range(C):
          x  = X_validation[Y_validation==i+1]
          a3 = A3[j,Y_validation==i+1]
          ax.scatter3D(x.T[0],x.T[1],a3,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3

      p=3
      Z1 = W1.dot(X_test.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = softmax(Z3)
      Colour=["red","green","blue"]
      for j in range(C):
        ax = plt.subplot(C,3,p,projection="3d")
        plt.title("Test Data\nOutput Node"+str(j+1))
        for i in range(C):
          x  = X_test[Y_test==i+1]
          a3 = A3[j,Y_test==i+1]
          ax.scatter3D(x.T[0],x.T[1],a3,color=Colour[i],label="Class "+str(i+1))
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        plt.legend(loc=1,fontsize="small")
        p+=3
      #plt.tight_layout()
      plt.show()

      return W1, b1, W2, b2, W3, b3 , Z1, Z2, Z3, A1, A2, A3

In [ ]:
W1, b1, W2, b2, W3, b3 , Z1, Z2, Z3, A1, A2, A3 = gradient_descent(LS_X_train, LS_y_train,LS_X_validation,LS_y_validation,LS_X_test,LS_y_test,
                                  alpha=0.01,Error=1e-5,iterations=1e3,node1=10,node2=5,title="Non Linear Seperable Data")


In [ ]:
W1, b1, W2, b2, W3, b3 , Z1, Z2, Z3, A1, A2, A3 = gradient_descent(NLS_X_train, NLS_y_train,NLS_X_validation,NLS_y_validation,NLS_X_test,NLS_y_test,
                                  alpha=0.001,Error=1e-5,iterations=1e2,node1=100,node2=50,title="Non Linear Seperable Data")


##Regression

###1 hidden layer

In [ ]:
def init_params(d,nodes):
    W1 = np.random.normal(0,1,(nodes, d))
    b1 = np.random.normal(0,1,(nodes, 1))
    W2 = np.random.normal(0,1,(1, nodes))
    b2 = np.random.normal(0,1,(1, 1))
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def linear(Z):
    return Z

def forward_prop(W1, b1, W2, b2, X,d):
    X = np.reshape(X, (d,1))
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = linear(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y,d,nodes):
    X = np.reshape(X,(d,1))
    Y = np.reshape(Y,(1,1))
    A2 = np.reshape(A2,(1,1))
    dZ2 = A2 - Y
    dZ2 = np.reshape(dZ2, (1, 1))
    dW2 = dZ2.dot(A1.T)
    dW2 = np.reshape(dW2, (1, nodes))
    db2 = np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = dZ1.dot(X.T)
    dW1 = np.reshape(dW1,(nodes,d))
    db1 = np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2

    return W1, b1, W2, b2

In [ ]:
def gradient_descent(X, Y,X_validation,Y_validation,X_test,Y_test,iterations,alpha,Error,nodes,title):
    m = X.shape[0]
    try:d = X.shape[1]
    except: d = 1
    Error_train      = []
    Error_validation = []
    Error_test       = []
    W1, b1, W2, b2 = init_params(d,nodes)
    for i in range(int(iterations)):
      for j in range(m):
        k = np.random.randint(0,m-1)
        tempX = X[k]
        tempY = Y[k]

        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, tempX,d)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, tempX, tempY,d,nodes)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)


      X1 = np.reshape(X,(m,d))
      Z1 = W1.dot(X1.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = linear(Z2)
      A2 = A2.reshape(m)
      y_train_pred = A2
      Error_train.append(((A2-Y)**2).sum()/len(X1))

      X1 = np.reshape(X_validation,(len(X_validation),d))
      Z1 = W1.dot(X1.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = linear(Z2)
      A2 = A2.reshape(len(X_validation))
      y_validation_pred = A2
      Error_validation.append(((A2-Y_validation)**2).sum()/len(X1))

      X1 = np.reshape(X_test,(len(X_test),d))
      Z1 = W1.dot(X1.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = linear(Z2)
      A2 = A2.reshape(len(X_test))
      y_test_pred = A2
      Error_test.append(((A2-Y_test)**2).sum()/len(X1))

      if i>1:
        if abs(Error_train[-1]-Error_train[-2]) and abs(Error_train[-1]-Error_train[-3]) <Error:
          break
    #Error Plot
    plt.figure(figsize=(12,5))
    plt.suptitle(title,fontweight="bold",y=1+1e-2)
    plt.subplot(1,3,1)
    plt.title("Training Error")
    plt.plot(range(1,len(Error_train)+1),Error_train,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()

    plt.subplot(1,3,2)
    plt.title("Validation Error")
    plt.plot(range(1,len(Error_validation)+1),Error_validation,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()
    #plt.tight_layout()


    plt.subplot(1,3,3)
    plt.title("Test Error")
    plt.plot(range(1,len(Error_test)+1),Error_test,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12,5))
    plt.suptitle(title,fontweight="bold",y=1+1e-2)
    bar_x = ("Training Data","Validation Data","Test Data")
    bar_y = (Error_train[-1],Error_validation[-1],Error_test[-1])
    plt.bar(bar_x,bar_y)
    plt.xlabel("Data")
    plt.ylabel("Mean Square Error")
    plt.grid()
    plt.tight_layout()
    plt.show()

    #Actual vs Predicted Output
    plt.figure(figsize=(12,5))
    plt.suptitle(title,fontweight="bold",y=1+1e-2)
    #Training Data
    plt.subplot(1,3,1)
    plt.title("Training Data")
    plt.scatter(Y,y_train_pred,marker=".")
    plt.xlabel("Target Output")
    plt.ylabel("Model Output")
    plt.grid()
    #Validation Data
    plt.subplot(1,3,2)
    plt.title("Validation Data")
    plt.scatter(Y_validation,y_validation_pred,marker=".")
    plt.xlabel("Target Output")
    plt.ylabel("Model Output")
    plt.grid()
    #Test Data
    plt.subplot(1,3,3)
    plt.title("Training Data")
    plt.scatter(Y_test,y_test_pred,marker=".")
    plt.xlabel("Target Output")
    plt.ylabel("Model Output")
    plt.grid()
    plt.tight_layout()
    plt.show()

    #Plot Hidden Layer
    plt.figure(figsize=(12.5,4*nodes))
    plt.suptitle(title+"\nHidden Layer",fontweight="bold",y=1+1e-2)

    p = 1
    X1 = np.reshape(X,(m,d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = linear(Z2)
    A2 = A2.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Training Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A1[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Training Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A1[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 2
    X1 = np.reshape(X_validation,(len(X_validation),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = linear(Z2)
    A2 = A2.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Validation Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A1[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Validation Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A1[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 3
    X1 = np.reshape(X_test,(len(X_test),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = linear(Z2)
    A2 = A2.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Test Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A1[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Test Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A1[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3
    plt.tight_layout()
    plt.show()

    #Plot Output Layer
    nodes=1
    plt.figure(figsize=(12.5,5*nodes))
    plt.suptitle(title+"\nOutput Layer",fontweight="bold",y=1+1e-2)

    p = 1
    X1 = np.reshape(X,(m,d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = linear(Z2)
    A2 = A2.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Training Data\nOutput Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A2)
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Training Data\nOutput Node"+str(j+1))
        plt.scatter(X1,A2)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 2
    X1 = np.reshape(X_validation,(len(X_validation),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = linear(Z2)
    A2 = A2.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Validation Data\nOutput Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A2)
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Validation Data\nOutput Node"+str(j+1))
        plt.scatter(X1,A2)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 3
    X1 = np.reshape(X_test,(len(X_test),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = linear(Z2)
    A2 = A2.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Test Data\nOutput Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A2)
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Test Data\nOutput Node"+str(j+1))
        plt.scatter(X1,A2)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3
    plt.tight_layout()
    plt.show()
    return (y_train_pred,y_validation_pred,y_test_pred)

In [ ]:
Univariate_y_pred = gradient_descent(Univariate_X[0], Univariate_y[0],Univariate_X[1], Univariate_y[1],Univariate_X[2], Univariate_y[2],
                                  alpha=.01,Error=1e-5,iterations=1e3,nodes=40,title="Univariate Data")

In [ ]:
Bivariate_y_pred = gradient_descent(Bivariate_X[0], Bivariate_y[0],Bivariate_X[1], Bivariate_y[1],Bivariate_X[2], Bivariate_y[2],
                                  alpha=.01,Error=1e-5,iterations=1e3,nodes=20,title="Bivariate Data")

In [ ]:
#Plot
plt.figure(figsize=(12,12.5))
plt.suptitle("Actual vs Predicted Output",fontweight="bold",y=1+1e-2)
plt.subplot(3,2,1)
plt.title("Univariate Train Data")
plt.scatter(Univariate_X[0],Univariate_y[0],label='Actual',alpha=1)
plt.scatter(Univariate_X[0],Univariate_y_pred[0],label='Prediction',alpha=1/3)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

plt.subplot(3,2,3)
plt.title("Univariate Validation Data")
plt.scatter(Univariate_X[1],Univariate_y[1],label='Actual',alpha=1)
plt.scatter(Univariate_X[1],Univariate_y_pred[1],label='Prediction',alpha=1/3)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

plt.subplot(3,2,5)
plt.title("Univariate Test Data")
plt.scatter(Univariate_X[2],Univariate_y[2],label='Actual',alpha=1)
plt.scatter(Univariate_X[2],Univariate_y_pred[2],label='Prediction',alpha=1/3)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,2,projection="3d")
plt.title("Bivariate Train Data")
ax.scatter3D(*Bivariate_X[0].T,Bivariate_y_pred[0],label='Prediction',alpha=1)
ax.scatter3D(*Bivariate_X[0].T,Bivariate_y[0],label='Actual',alpha=1/3)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,4,projection="3d")
plt.title("Bivariate Validation Data")
ax.scatter3D(*Bivariate_X[1].T,Bivariate_y_pred[1],label='Prediction',alpha=1)
ax.scatter3D(*Bivariate_X[1].T,Bivariate_y[1],label='Actual',alpha=1/3)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,6,projection="3d")
plt.title("Bivariate Test Data")
ax.scatter3D(*Bivariate_X[2].T,Bivariate_y[2],label='Actual',alpha=1)
ax.scatter3D(*Bivariate_X[2].T,Bivariate_y_pred[2],label='Prediction',alpha=1/3)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)
plt.tight_layout()
plt.show()

### 2 hidden layer


In [ ]:
def init_params(d,node1,node2):
    W1 = np.random.normal(0,1,(node1, d))
    b1 = np.random.normal(0,1,(node1, 1))
    W2 = np.random.normal(0,1,(node2, node1))
    b2 = np.random.normal(0,1,(node2, 1))
    W3 = np.random.normal(0,1,(1, node2))
    b3 = np.random.normal(0,1,(1, 1))
    return W1, b1, W2, b2, W3, b3


def ReLU(Z):
    return np.maximum(Z, 0)

def linear(Z):
    return Z

def forward_prop(W1, b1, W2, b2, W3, b3, X, d):
    X = np.reshape(X, (d,1))
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def ReLU_deriv(Z):
    return Z > 0

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y,d,node1,node2):

    X = np.reshape(X,(d,1))
    Y = np.reshape(Y,(1,1))
    A3 = np.reshape(A3,(1,1))
    dZ3 = A3 - Y
    dZ3 = np.reshape(dZ3, (1, 1))
    dW3 = dZ3.dot(A2.T)
    db3 = np.sum(dZ3)
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = dZ2.dot(A1.T)
    db2 = np.sum(dZ2)

    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = dZ1.dot(X.T)
    db1 = np.sum(dZ1)
    return dW1, db1, dW2, db2, dW3, db3


def update_params(W1, b1, W2, b2,W3,b3, dW1, db1, dW2, db2,dW3,db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    return W1, b1, W2, b2, W3, b3

In [ ]:
def gradient_descent(X, Y,X_validation,Y_validation,X_test,Y_test,iterations,alpha,Error,node1,node2,title):
    m = X.shape[0]
    try:d = X.shape[1]
    except: d = 1
    Error_train      = []
    Error_validation = []
    Error_test       = []
    W1, b1, W2, b2,W3,b3 = init_params(d,node1,node2)
    for i in range(int(iterations)):
      for j in range(m):
        k = np.random.randint(0,m-1)
        tempX = X[k]
        tempY = Y[k]

        Z1, A1, Z2, A2,Z3,A3 = forward_prop(W1, b1, W2, b2,W3,b3, tempX,d)
        dW1, db1, dW2, db2,dW3,db3 = backward_prop(Z1, A1, Z2, A2,Z3,A3, W1, W2,W3, tempX, tempY,d,node1,node2)
        W1, b1, W2, b2,W3,b3 = update_params(W1, b1, W2, b2,W3,b3, dW1, db1, dW2, db2,dW3,db3, alpha)


      X1 = np.reshape(X,(m,d))
      Z1 = W1.dot(X1.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = linear(Z3)
      A3 = A3.reshape(len(X1))
      y_train_pred = A3
      Error_train.append(((A3-Y)**2).sum()/len(X1))

      X1 = np.reshape(X_validation,(len(X_validation),d))
      Z1 = W1.dot(X1.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = linear(Z3)
      A3 = A3.reshape(len(X1))
      y_validation_pred = A3
      Error_validation.append(((A3-Y_validation)**2).sum()/len(X1))

      X1 = np.reshape(X_test,(len(X_test),d))
      Z1 = W1.dot(X1.T) + b1
      A1 = ReLU(Z1)
      Z2 = W2.dot(A1) + b2
      A2 = ReLU(Z2)
      Z3 = W3.dot(A2) + b3
      A3 = linear(Z3)
      A3 = A3.reshape(len(X1))
      y_test_pred = A3
      Error_test.append(((A3-Y_test)**2).sum()/len(X1))

      if i>1:
        if abs(Error_train[-1]-Error_train[-2]) and abs(Error_train[-1]-Error_train[-3]) <Error:
          break
    #Error Plot
    plt.figure(figsize=(12,5))
    plt.suptitle(title,fontweight="bold",y=1+1e-2)
    plt.subplot(1,3,1)
    plt.title("Training Error")
    plt.plot(range(1,len(Error_train)+1),Error_train,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()

    plt.subplot(1,3,2)
    plt.title("Validation Error")
    plt.plot(range(1,len(Error_validation)+1),Error_validation,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()
    #plt.tight_layout()


    plt.subplot(1,3,3)
    plt.title("Test Error")
    plt.plot(range(1,len(Error_test)+1),Error_test,marker=".")
    plt.xlabel("Epochs")
    plt.ylabel("Average Error")
    plt.grid()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12,5))
    plt.suptitle(title,fontweight="bold",y=1+1e-2)
    bar_x = ("Training Data","Validation Data","Test Data")
    bar_y = (Error_train[-1],Error_validation[-1],Error_test[-1])
    plt.bar(bar_x,bar_y)
    plt.xlabel("Data")
    plt.ylabel("Mean Square Error")
    plt.grid()
    plt.tight_layout()
    plt.show()

    #Actual vs Predicted Output
    plt.figure(figsize=(12,5))
    plt.suptitle(title,fontweight="bold",y=1+1e-2)
    #Training Data
    plt.subplot(1,3,1)
    plt.title("Training Data")
    plt.scatter(Y,y_train_pred,marker=".")
    plt.xlabel("Target Output")
    plt.ylabel("Model Output")
    plt.grid()
    #Validation Data
    plt.subplot(1,3,2)
    plt.title("Validation Data")
    plt.scatter(Y_validation,y_validation_pred,marker=".")
    plt.xlabel("Target Output")
    plt.ylabel("Model Output")
    plt.grid()
    #Test Data
    plt.subplot(1,3,3)
    plt.title("Training Data")
    plt.scatter(Y_test,y_test_pred,marker=".")
    plt.xlabel("Target Output")
    plt.ylabel("Model Output")
    plt.grid()
    plt.tight_layout()
    plt.show()

    #Plot Hidden Layer1
    plt.figure(figsize=(12.5,4*node1))
    plt.suptitle(title+"\nHidden Layer",fontweight="bold",y=1+1e-2)

    p = 1
    X1 = np.reshape(X,(m,d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(node1):
      if d==2:
        ax = plt.subplot(node1,3,p,projection="3d")
        plt.title("Training Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A1[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(node1,3,p)
        plt.title("Training Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A1[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 2
    X1 = np.reshape(X_validation,(len(X_validation),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(node1):
      if d==2:
        ax = plt.subplot(node1,3,p,projection="3d")
        plt.title("Validation Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A1[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(node1,3,p)
        plt.title("Validation Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A1[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 3
    X1 = np.reshape(X_test,(len(X_test),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(node1):
      if d==2:
        ax = plt.subplot(node1,3,p,projection="3d")
        plt.title("Test Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A1[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(node1,3,p)
        plt.title("Test Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A1[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3
    plt.tight_layout()
    plt.show()

    #Plot Hidden Layer2
    plt.figure(figsize=(12.5,4*node2))
    plt.suptitle(title+"\nHidden Layer",fontweight="bold",y=1+1e-2)

    p = 1
    X1 = np.reshape(X,(m,d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(node2):
      if d==2:
        ax = plt.subplot(node2,3,p,projection="3d")
        plt.title("Training Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A2[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(node2,3,p)
        plt.title("Training Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A2[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 2
    X1 = np.reshape(X_validation,(len(X_validation),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(node2):
      if d==2:
        ax = plt.subplot(node2,3,p,projection="3d")
        plt.title("Validation Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A2[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(node2,3,p)
        plt.title("Validation Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A2[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 3
    X1 = np.reshape(X_test,(len(X_test),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(node2):
      if d==2:
        ax = plt.subplot(node2,3,p,projection="3d")
        plt.title("Test Data\nHidden Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A2[j])
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(node2,3,p)
        plt.title("Test Data\nHidden Node"+str(j+1))
        plt.scatter(X1,A2[j])
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3
    plt.tight_layout()
    plt.show()

    #Plot Output Layer
    nodes=1
    plt.figure(figsize=(12.5,5*nodes))
    plt.suptitle(title+"\nOutput Layer",fontweight="bold",y=1+1e-2)

    p = 1
    X1 = np.reshape(X,(m,d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Training Data\nOutput Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A3)
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Training Data\nOutput Node"+str(j+1))
        plt.scatter(X1,A3)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 2
    X1 = np.reshape(X_validation,(len(X_validation),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Validation Data\nOutput Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A3)
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Validation Data\nOutput Node"+str(j+1))
        plt.scatter(X1,A3)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3

    p = 3
    X1 = np.reshape(X_test,(len(X_test),d))
    Z1 = W1.dot(X1.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = linear(Z3)
    A3 = A3.reshape(len(X1))
    for j in range(nodes):
      if d==2:
        ax = plt.subplot(nodes,3,p,projection="3d")
        plt.title("Test Data\nOutput Node"+str(j+1))
        ax.scatter3D(X1.T[0],X1.T[1],A3)
        plt.xlabel("x1")
        plt.ylabel("x2")
        ax.set_zlabel("Activation for Neuron "+str(j+1))
        plt.grid()
        p+=3
      elif d==1:
        plt.subplot(nodes,3,p)
        plt.title("Test Data\nOutput Node"+str(j+1))
        plt.scatter(X1,A3)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.grid()
        p+=3
    plt.tight_layout()
    plt.show()
    return (y_train_pred,y_validation_pred,y_test_pred)

In [ ]:
Univariate_y_pred = gradient_descent(Univariate_X[0], Univariate_y[0],Univariate_X[1], Univariate_y[1],Univariate_X[2], Univariate_y[2],
                                  alpha=.001,Error=1e-5,iterations=1e3,node1=40,node2=20,title="Univariate Data")

In [ ]:
Bivariate_y_pred = gradient_descent(Bivariate_X[0], Bivariate_y[0],Bivariate_X[1], Bivariate_y[1],Bivariate_X[2], Bivariate_y[2],
                                  alpha=.001,Error=1e-5,iterations=1e3,node1=20,node2=10,title="Bivariate Data")

In [ ]:
#Plot
plt.figure(figsize=(12,12.5))
plt.suptitle("Actual vs Predicted Output",fontweight="bold",y=1+1e-2)
plt.subplot(3,2,1)
plt.title("Univariate Train Data")
plt.scatter(Univariate_X[0],Univariate_y[0],label='Actual',alpha=1)
plt.scatter(Univariate_X[0],Univariate_y_pred[0],label='Prediction',alpha=1/3)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

plt.subplot(3,2,3)
plt.title("Univariate Validation Data")
plt.scatter(Univariate_X[1],Univariate_y[1],label='Actual',alpha=1)
plt.scatter(Univariate_X[1],Univariate_y_pred[1],label='Prediction',alpha=1/3)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

plt.subplot(3,2,5)
plt.title("Univariate Test Data")
plt.scatter(Univariate_X[2],Univariate_y[2],label='Actual',alpha=1)
plt.scatter(Univariate_X[2],Univariate_y_pred[2],label='Prediction',alpha=1/3)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,2,projection="3d")
plt.title("Bivariate Train Data")
ax.scatter3D(*Bivariate_X[0].T,Bivariate_y_pred[0],label='Prediction',alpha=1)
ax.scatter3D(*Bivariate_X[0].T,Bivariate_y[0],label='Actual',alpha=1/3)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,4,projection="3d")
plt.title("Bivariate Validation Data")
ax.scatter3D(*Bivariate_X[1].T,Bivariate_y_pred[1],label='Prediction',alpha=1)
ax.scatter3D(*Bivariate_X[1].T,Bivariate_y[1],label='Actual',alpha=1/3)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)

ax = plt.subplot(3,2,6,projection="3d")
plt.title("Bivariate Test Data")
ax.scatter3D(*Bivariate_X[2].T,Bivariate_y[2],label='Actual',alpha=1)
ax.scatter3D(*Bivariate_X[2].T,Bivariate_y_pred[2],label='Prediction',alpha=1/3)
plt.xlabel("x1")
plt.ylabel("x2")
ax.set_zlabel("y")
plt.grid()
plt.legend(loc=1)
plt.tight_layout()
plt.show()